# mlflow - autolog

## 1.환경준비

### (1) 라이브러리 Import

* 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler

import mlflow

### (2) 데이터 불러오기

* 카시트 판매량 데이터

|	변수명	|	설명	|	구분	|
|----|----|----|
|	**Sales** 	|	 **각 지역 판매량(단위 : 1000개)**	|	**Target**	|
|	CompPrice 	|	지역별 경쟁사 판매가격(달러)	|	feature	|
|	Income 	|	가구당 평균 소득액(1000달러)	|	feature	|
|	Advertising 	|	 각 지역, 회사의 광고 예산(1000달러)	|	feature	|
|	Population 	|	 지역 인구수(단위 : 1000명)	|	feature	|
|	Price 	|	 자사 지역별 판매가격(달러)	|	feature	|
|	ShelveLoc 	|	 진열상태(범주 : Bad, Medium, Good)	|	feature	|
|	Age 	|	 지역 인구의 평균 연령	|	feature	|
|	Education 	|	 교육수준(범주 : 10~18)	|	feature	|
|	Urban 	|	 매장이 도심에 있는지 여부(범주 : Yes, No)	|	feature	|
|	US 	|	 매장이 미국에 있는지 여부(범주 : Yes, No)	|	feature	|


* 데이터 경로 : https://raw.githubusercontent.com/DA4BAM/dataset/master/Carseats.csv

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/Carseats.csv'
carseat = pd.read_csv(path)
carseat.head()

## 2.전처리 및 추적 준비

### (1) 데이터 전처리

* 데이터 준비

In [ ]:
target = 'Sales'
x = carseat.drop(target, axis=1)
y = carseat.loc[:, target]

* 가변수화

In [ ]:
cat_cols = {'ShelveLoc':['Bad','Medium','Good']
            , 'Education':[10,11,12,13,14,15,16,17,18]
            , 'US':['No','Yes']
            , 'Urban':['No','Yes']}

for var, values in cat_cols.items():
    x[var] = pd.Categorical(x[var], categories = values)

x = pd.get_dummies(x, columns = cat_cols.keys(), drop_first = True)    

* 데이터분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2, random_state = 20)

* 스케일링

In [ ]:
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

### (2) Tracking 준비

#### 1) 추적설정

In [ ]:
mlflow_uri = "sqlite:///mlflow.db"
mlflow.set_tracking_uri(mlflow_uri)

#### 2) 새 실험 만들기

In [ ]:
exp_id = mlflow.create_experiment("exp3")
exp_id

## 3.autolog 1

### (1) sklearn 알고리즘

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

#### 1) 수동 로깅

In [ ]:
with mlflow.start_run(experiment_id = exp_id):
    
    model = RandomForestRegressor()
    model.fit(x_train, y_train)
    
    pred = model.predict(x_val)
    mlflow.log_metric("mae", mean_absolute_error(y_val, pred))
        
    # best 모델 기록하기
    mlflow.sklearn.log_model(model, "model", registered_model_name="RF")

#### 2) autolog

In [ ]:
with mlflow.start_run(experiment_id=exp_id):
    
    # autolog!
    mlflow.sklearn.autolog()
    
    model = RandomForestRegressor()
    model.fit(x_train, y_train)
    
    mlflow.sklearn.autolog(disable = True)

* mlflow ui에서 두 결과의 차이를 비교해 봅시다.

### (2) Grid Search 튜닝

#### 1)튜닝 일반 코드

In [ ]:
# 1.튜닝 ----------------------------------------------
## 1) 하이퍼파라미터 범위 설정
params = {'n_estimators':range(20,201,20)}

## 2) GridSearchCV 모델 선언
model = GridSearchCV(RandomForestRegressor(), params, cv = 5)

## 3) 학습(튜닝)
model.fit(x_train, y_train)

# 2.튜닝 결과 데이터프레임을 csv로 저장하기
result = pd.DataFrame(model.cv_results_)
result[['param_n_estimators', 'mean_test_score','rank_test_score']].to_csv('result.csv', index = False)

In [ ]:
# 3.튜닝 결과 그래프
plt.plot('param_n_estimators', 'mean_test_score', data = result, marker = '.' )
plt.title('RF GridSearch Tuning')
plt.ylabel('r2 score')
plt.xlabel('n_estimators')
plt.grid()    
plt.show()

In [ ]:
# 가장 성능이 좋았을 때 하이퍼파라미터 값
model.best_params_

In [ ]:
# 가장 성능이 좋았을 때의 성능
model.best_score_

In [ ]:
# 가장 성능이 좋은 모델
model.best_estimator_

#### 2)autolog

In [ ]:
with mlflow.start_run(experiment_id=exp_id):
    
    mlflow.sklearn.autolog()
    
    params = {'n_estimators':range(20,201,20)}
    model = GridSearchCV(RandomForestRegressor(), params, cv = 5)
    model.fit(x_train, y_train)
    
    # 자동기록되는 부분 : 하이퍼파리미터, 성능지표, 성능평가 결과(.cv_results_)
    
    # 수동으로 등록해야 하는 부분 : 모델등록, 그래프, 추가적인 fitting 함수
    result = pd.DataFrame(model.cv_results_)
    plt.plot('param_n_estimators', 'mean_test_score', data = result, marker = '.' )
    plt.title('RF GridSearch Tuning')
    plt.ylabel('r2 score')
    plt.xlabel('n_estimators')
    plt.grid()    
    plt.savefig('RF_GridSearch_Tuning.png')
    mlflow.log_artifact("RF_GridSearch_Tuning.png")
    
    # best 모델 기록하기
    mlflow.sklearn.log_model(model.best_estimator_, "model", registered_model_name="RF_Tuning")    
       
    mlflow.sklearn.autolog(disable = True)

### (3) 실습
* exp3에 KNN regressor 튜닝 기록하기
    * mlflow.start_run 함수에 experiment_id=exp_id, run_name = 'gridsearch_tuning01' 이라고 실행 이름을 지정해 봅시다.
    * 하이퍼퍼라미터 범위 
        * n_neighbors : 1~50
        * metric : ‘euclidean’, ‘manhattan’
    * autolog
    * 추가 로깅
        * 모델 : best_estimator_ 
        * 그래프 : metric별 n_neighbors에 따른 성능 추이 그래프
* 먼저 튜닝을 시도해보고, 이상이 없으면 mlflow에 logging 하도록 합시다.
* 기록 후 mlflow 관리 페이지에서 확인해 봅시다.

In [ ]:
with mlflow.start_run(experiment_id=exp_id, run_name = 'gridsearch_tuning01'):
    
    # autolog 시작!

    
    # 튜닝 코드

    
        
    # 수동등록 : 그래프
    
    
    # 수동등록 : best 모델 기록하기
    
    
    # autolog 종료


## 4.autolog2 : Keras(Deep Learning)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.backend import clear_session
from keras.callbacks import Callback
from keras.optimizers import Adam

import mlflow.keras

In [ ]:
# 학습곡선 그래프
def dl_history_plot(history):
    plt.figure(figsize=(8,5))
    plt.plot(history['loss'], label='train_err', marker = '.')
    plt.plot(history['val_loss'], label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

### (1) Keras 모델링

* input_shape 확인

In [ ]:
nfeatures = x_train.shape[1] #num of columns
nfeatures

* 모델 구조 설계

In [ ]:
model = Sequential([ Dense(1, input_shape = (nfeatures,))  ])
model.summary()

* 컴파일 및 학습

In [ ]:
model.compile(optimizer= Adam(learning_rate = 0.1), loss='mse')
history = model.fit(x_train_s, y_train, epochs=30, validation_split=.2).history

* 학습 곡선

In [ ]:
dl_history_plot(history)

### (2) autolog

In [ ]:
with mlflow.start_run(experiment_id=exp_id):
    
    # autolog 시작
    mlflow.keras.autolog()
    
    # 모델 설계 및 컴파일
    model = Sequential([ Dense(1, input_shape = (nfeatures,))  ])
    model.compile(optimizer= Adam(learning_rate = 0.1), loss='mse')

    # 학습
    history = model.fit(x_train_s, y_train, epochs=30, validation_split=.2).history
    
    # 모델등록
    mlflow.keras.log_model(model, "keras-model",  registered_model_name="DL")
    
    # autolog 종료
    mlflow.keras.autolog(disable = True)

* 딥러닝모델에서 로깅된 내용을 확인해봅시다.
* 어떤 정보를 더 로깅해야 할까요?

## 6.실습
* 실습을 해 봅시다.

### (1) 데이터 준비

* 데이터 로딩

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/credit_all.csv'
data = pd.read_csv(path)
data.loc[data['Payment'] == 4, 'Payment'] = 3
data.drop(['Telephone','ForeignWorker'], axis = 1, inplace = True)
data.head()

| 칼럼명 | 설명 | | 값 의미 |  
|-----|-----|-----|-----|
|	Creditability	|	Creditability(Target)	|	향후 신용도	|	0 : Bad, 1 : Good	|
|	AccountBalance	|	Account Balance	|	은행잔고	|	1: No account, 2 : None (No balance), 3 : Some Balance	|
|	CreditDuration	|	Duration of Credit (month)	|	신청한 대출기간(월)	|	숫자	|
|	Payment	|	Payment Status of Previous Credit	|	과거 대출 납입 상태	|	0 : 연체, 1 : 기타신용, 2 : 완납, 3 : 정상 대출상환 중 |
|	Purpose	|	Purpose	|	신청한 대출목적	|	1 : New Car , 2 : Used Car , 3 : Furniture , 4 : TV , 5 : Appliances , 6 : Repair , 8 : Vacation , 9 :Retraining , 10 : Business , 0 : Other	|
|	CreditAmount	|	Credit Amount($)	|	신청한 대출금액	|		|
|	Employment	|	Length of current employment(Month)	|	현 직업 근무 기간	|	1: Unemployed,  2: <1 Year,  3: [1, 4),  4: [4, 7),  5: Above 7	|
|	SexMarital	|	Sex & Marital Status	|	성별 & 결혼상태	|	1: Male, Divorced, 2: Male, Single , 3: Male, Married/Widowed , 4: Female	|
|	CurrentAddress	|	Duration in Current address	|	현 거주지 거주기간	|	1: <1 Year , 2: [1, 4) , 3: [4, 7) , 4: Above 7	|
|	MostValuable	|	Most valuable available asset	|	가장 가치있는 자산	|	1: None , 2: Car , 3: Life Insurance , 4: Real Estate	|
|	Age	|	Age (years)	|	나이	|		|
|	AppartmentType	|	Type of apartment	|	주거환경	|	1: free apartment, 2: Rented, 3: Owned	|
|	NoCredits	|	No of Credits at this Bank	|	현재 총 대출 건수	|	1 : one, 2 : 2 ~ 3, 3 : 4 ~ 5, 4 : 6 ~	|
|	Occupation	|	Occupation	|	직업	|	1: Unemployed, unskilled, 2: Unskilled Permanent Resident, 3: Skilled, 4: Executive	|


* x, y 나누기

In [ ]:
target = 'Creditability'
x = data.drop(target, axis = 1)
y = data.loc[:,target]

* 가변수화

In [ ]:
dummy_vars = [ 'Payment', 'Purpose', 'SexMarital', 'MostValuableAsset']
x = pd.get_dummies(x, columns = dummy_vars)
x.head()

* train : val:test 분할

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 20, random_state = 20)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = .2, random_state = 20)

* 스케일링

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)
x_test_s = scaler.transform(x_test)

x_train_s = pd.DataFrame(x_train_s, columns = list(x_train))
x_val_s = pd.DataFrame(x_val_s, columns = list(x_train))
x_test_s = pd.DataFrame(x_test_s, columns = list(x_train))

### (2) sklearn 튜닝 모델 로깅
* KNN, SVM, Decision Tree, RandomForest 등 알고리즘으로 모델 튜닝
* mlflow.start_run(experiment_id = exp2, run_name = `모델링의 특성이 반영된 이름`)
* autolog로 로깅
* 튜닝된 모델은 수동 로깅
    * 모델 이름 : `Credit`

In [ ]:
from sklearn.tree import DecisionTreeClassifier

### (3) Keras 모델 로깅
* mlflow.start_run(experiment_id = exp2, run_name = `모델링의 특성이 반영된 이름`)
* autolog로 로깅
* 튜닝된 모델은 수동 로깅
    * 모델 이름 : `Credit`

In [ ]:
nfeatures = x_train.shape[1] #num of columns
nfeatures

### (5) test 데이터 저장

In [ ]:
import joblib

joblib.dump(x_test, 'x_test.pkl')
joblib.dump(x_test_s, 'x_test_s.pkl')
joblib.dump(y_test, 'y_test.pkl')